In [1]:
import pandas as pd
import numpy as np

In [2]:
path = "/Users/blazdobravec/Documents/WORK/INTERNI-PROJEKTI/02_DEV_PROJECTS/tarifni_sistem_si_web_app/data/6-113167-15minMeritve2023-01-01-2023-12-31_leto2023.xlsx"

df = pd.read_excel(path, engine='openpyxl')

/Users/blazdobravec/opt/miniconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
df['p'] = df['P+ Prejeta delovna moč'] - df['P- Oddana delovna moč']
df['q'] = df['Q+ Prejeta jalova moč'] - df['Q- Oddana jalova moč']
df['a'] = df['Energija A+'] - df['Energija A-']

df.rename(columns={'Blok': 'block'}, inplace=True)
df_sett = df[['block', 'p', 'q', 'a']]
# create a month column
df_sett['month'] = df['Časovna značka'].dt.month
df_sett['year'] = df['Časovna značka'].dt.year
# create a year_month column that combines year and month with a -
df_sett['year_month'] = df_sett['year'].astype(str) + '-' + df_sett['month'].astype(str)

/var/folders/5l/_0j09j5n0gg7s_98h1l1nvw40000gn/T/ipykernel_18968/1619829615.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sett['month'] = df['Časovna značka'].dt.month
/var/folders/5l/_0j09j5n0gg7s_98h1l1nvw40000gn/T/ipykernel_18968/1619829615.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sett['year'] = df['Časovna značka'].dt.year
/var/folders/5l/_0j09j5n0gg7s_98h1l1nvw40000gn/T/ipykernel_18968/1619829615.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [4]:
df_sett

,block,p,q,a,month,year,year_month
0,4,1.104,NaN,0.276,1,2023,2023-1
1,4,2.680,NaN,0.670,1,2023,2023-1
2,4,0.776,NaN,0.194,1,2023,2023-1
3,4,0.204,NaN,0.051,1,2023,2023-1
4,4,0.180,NaN,0.045,1,2023,2023-1
...,...,...,...,...,...,...,...
35035,4,0.768,NaN,0.192,12,2023,2023-12
35036,4,0.764,NaN,0.191,12,2023,2023-12
35037,4,0.828,NaN,0.207,12,2023,2023-12
35038,4,0.904,NaN,0.226,12,2023,2023-12


In [5]:
bloki_cene_moci = np.array([0.24923, 0.04877, 0.01103, 0.00038, 0.00000]) + np.array([3.36401, 0.83363, 0.18034, 0.01278, 0.00000])
bloki_cene_energije = np.array([0.00663, 0.00620, 0.00589, 0.00592, 0.00589]) + np.array([0.01295, 0.01224, 0.01248, 0.01246, 0.01258])
# penali = 

prikljucna_moc = 14
skupina_koncnih_odjemalcev = 0
st_faz = 3
samooskrba = True
Fex = 0.9  # faktor presežene obračunske moči
VAT = 1.22

In [6]:

def energija_omreznine(df, bloki_cene_energije, placilo_omr_e):
    cene_e = placilo_omr_e
    df_energy = df.loc[df.a > 0]
    df_energy = df_energy[["a", "year_month", "block"]].groupby(['year_month', 'block']).sum()
    for year_month, vals in df_energy.groupby(level=0):
        month, _ = year_month.split("-")
        df = vals.droplevel(0)
        if 1 not in df.index:
            # insert it at the beginning
            df.loc[1] = 0
        if 5 not in df.index:
            df.loc[5] = 0 
        df.sort_index(inplace=True)
        # print(df)
        cena = df["a"] * bloki_cene_energije
        cene_e[int(month) - 1] = cena.sum()
    cene_e = np.array(cene_e)
    return cene_e

def moc_omreznine(df, bloki, bloki_cene_moci, placilo_omr_p):
    placilo_p = placilo_omr_p
    power = df[["p", "year_month", "block"]].groupby(['year_month', 'block']).max()
    for year_month, _ in power.groupby(level=0):
        year, month = year_month.split("-")
        if int(month) in [1,2,11,12]:
            cena = bloki * bloki_cene_moci * [1, 1, 1, 1, 0]
        else:
            cena = bloki * bloki_cene_moci * [0, 1, 1, 1, 1]
        placilo_p[int(month) - 1] = cena.sum()

    placilo_p = np.array(placilo_p)
    return placilo_p

from utils import handle_obr_moc, find_min_obr_p
def predlagane_obr_moci(prikljucna_moc, st_faz, skupina_koncnih_odjemalcev):
    blocks = []
    for block in range(1, 6):
        if skupina_koncnih_odjemalcev <= 1:
            selected_season = df_sett[df_sett.month.isin([1,2,11,12])]
            block_value = np.average(selected_season.loc[selected_season.block == block, "p"].nlargest(3))
        else:
            block_value = np.average(df_sett.loc[df_sett.block == block, "p"].nlargest(1))
        blocks.append(block_value)
    min_obr_p = find_min_obr_p(st_faz, prikljucna_moc)
    blocks = np.array(handle_obr_moc(blocks, prikljucna_moc, min_obr_p))
    # round
    blocks = np.round(blocks, 1)
    return blocks

def penali(df, bloki, Fex, cene_moci):
    df['adjusted_power'] = df.apply(lambda x: x['p'] - bloki[x['block'] - 1], axis=1)
    df = df[df.adjusted_power > 0]
    monthly_power_sum = df.groupby('month')['adjusted_power'].sum().reset_index()
    print(monthly_power_sum)
    
    # subtract the power from the power of the block
    # block_powers_penalties = np.zeros(5)
    # for i in range(len(powers_masked)):  # gre prek vseh blokov
    #     obr_power = bloki[i]
    #     powers_blok = powers_masked[i]
    #     power_exceeded = (powers_blok - obr_power) * (powers_blok
    #                                                     > obr_power)
    #     block_powers_penalties[i] = Fex * np.sqrt(sum(power_exceeded**2))

    # izracun penalov
    # powers_penalties = np.sum(block_powers_penalties * cene_moci)

In [7]:
import time

start = time.time()
placilo_omr_e = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
placilo_omr_p = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
if samooskrba:
    if df_sett.a.sum() < 0:
        # set cene_e to 0
        placilo_omr_e = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    else:
        placilo_omr_e = energija_omreznine(df_sett, bloki_cene_energije, placilo_omr_e)
    print("leto omreznina energija:", np.sum(placilo_omr_e)*VAT)

bloki = predlagane_obr_moci(prikljucna_moc, st_faz, skupina_koncnih_odjemalcev)
placilo_omr_p = moc_omreznine(df_sett, bloki, bloki_cene_moci, placilo_omr_p)
penali = penali(df_sett, bloki, Fex, bloki_cene_moci)

# calculate the excess power and penalties

print("leto omreznina moč:", np.sum(placilo_omr_p)*VAT)
print("sum:", (np.sum(placilo_omr_p)*VAT + np.sum(placilo_omr_e)*VAT))
end = time.time()

print("time:", end - start)

leto omreznina energija: 0.0
   month  adjusted_power
0      1           0.332
1     11           0.588
leto omreznina moč: 237.65540463999997
sum: 237.65540463999997
time: 0.1460890769958496


/var/folders/5l/_0j09j5n0gg7s_98h1l1nvw40000gn/T/ipykernel_18968/1118679770.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['adjusted_power'] = df.apply(lambda x: x['p'] - bloki[x['block'] - 1], axis=1)
